In [1]:
import pandas as pd
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from sklearn.decomposition import TruncatedSVD, PCA, IncrementalPCA, KernelPCA, SparsePCA
import numpy as np
import pickle

In [2]:
output_path = "./evaluation/"
items_path = "./data/games.pkl.gz"
data_path = "./data/interactions_splits_"
reviews_path = "./data/reviews.parquet"

In [3]:
evaluations = list()
use_data = ["specs", "genres", "tags", "early_access", "publisher"]
weighting_scheme = {"playtime": True, "sentiment": "mixed", "reviews": True}

rec = ImprovedRecommender(items_path, train_path=f"{data_path}0_train.parquet", test_path=f"{data_path}0_test.parquet",
                            val_path=f"{data_path}0_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, dim_red=None, columns=use_data, weighting_scheme=weighting_scheme)
rec.generate_recommendations(amount=20, silence=True, read_max=20)

rec.qualitative_evaluation()

,owned_items,recommended_items
35844,"[Uplink, Call of Duty: World at War, DEFCON, M...","[Operation Flashpoint: Red River, Red Orchestr..."
12778,"[Natural Selection 2, Dragon Age: Origins - Ul...","[Worms W.M.D, Mount & Blade, Knights of Pen an..."
52734,[The Expendabros],"[Broforce, Rocketbirds: Hardboiled Chicken, Fo..."
27853,"[Operation Flashpoint: Dragon Rising, Colin Mc...","[F1 2016, Tropico 5, Planet Coaster, F1 2014, ..."
45757,"[Just Cause, Guns of Icarus Online, Poker Nigh...","[Mount & Blade: Warband, Fallout 3, Saints Row..."
36398,"[Double Action: Boogaloo, Poker Night 2, Natur...","[Copa Petrobras de Marcas, Codename CURE, BLOC..."
561,[BIT.TRIP Presents... Runner2: Future Legend o...,"[The Escapists, METAL SLUG 3, Waveform, Polari..."
9562,"[Hacker Evolution, ""Glow Ball"" - The billiard ...","[RIFT, Mabinogi, Riders of Icarus, Dragons and..."
52541,[Eternal Senia],"[LiEat, Skyborn, Helen's Mysterious Castle, Th..."
37819,"[Bloons TD Battles, Sven Co-op, Boson X, Tacti...","[Tales from the Borderlands, The Walking Dead,..."
